In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,"Schumer, Toomey Reach Compromise On Fed Provis...",https://www.npr.org/2020/12/20/948505026/deal-...,"Sun, 20 Dec 2020 15:54:00 GMT",2020-12-20 15:54:00+00:00
6,COVID-19 Daily Update 12-20-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 20 Dec 2020 15:00:15 GMT",2020-12-20 15:00:15+00:00
4,Moderna vaccine shipments begin as US reels un...,https://www.theguardian.com/world/2020/dec/20/...,"Sun, 20 Dec 2020 13:29:00 GMT",2020-12-20 13:29:00+00:00
2,"United Kingdom Locks Down As Cases Of New, Mor...",https://www.npr.org/2020/12/20/948498254/uk-covid,"Sun, 20 Dec 2020 13:07:00 GMT",2020-12-20 13:07:00+00:00
1,Former COVID-19 Patient Thanks Hospital With 8...,https://www.npr.org/2020/12/20/948498247/forme...,"Sun, 20 Dec 2020 13:05:00 GMT",2020-12-20 13:05:00+00:00
7,CDC's vaccine advisers meet to plan next step ...,https://www.cnn.com/2020/12/20/health/covid-19...,"Sun, 20 Dec 2020 11:35:00 GMT",2020-12-20 11:35:00+00:00
5,UK health secretary says new COVID mutation is...,https://www.bostonglobe.com/2020/12/20/busines...,"Sun, 20 Dec 2020 05:59:00 GMT",2020-12-20 05:59:00+00:00
8,South Carolina Daily COVID-19 Update (December...,https://scdhec.gov/news-releases/south-carolin...,"Sat, 19 Dec 2020 16:11:22 GMT",2020-12-19 16:11:22+00:00
3,Dec. 19 update on COVID-19 in MN: December now...,https://www.mprnews.org/story/2020/12/19/lates...,"Sat, 19 Dec 2020 13:30:00 GMT",2020-12-19 13:30:00+00:00
9,Governor Lamont Implements New Guidelines for ...,https://portal.ct.gov/Office-of-the-Governor/N...,"Fri, 18 Dec 2020 22:07:31 GMT",2020-12-18 22:07:31+00:00
